# Prepare map for visualization
Steps:
- Convert to an equal area projection
- Define the color scheme as part of the geotiff

In [1]:
from osgeo import gdal
import sys
from pathlib import Path
sys.path.append(str(Path().absolute().parent))
import _functions as pmf

In [2]:
gdal.UseExceptions()

In [3]:
# Specify where the config file can be found
config_file = '../0_config/config.txt'

In [4]:
# Get the required info from the config file
data_path = pmf.read_from_config(config_file,'data_path')
tar_crs =  pmf.read_from_config(config_file,'topo_crs')

In [5]:
# Define where the map can be found
cluster_path = Path(data_path) / 'data' / 'geotiff_same_resolution' / 'res_0.01' / 'clustering' / 'res_0-01_mask-MERIT-GLCLU2019_datasets-29_reruns-10'
cluster_file = str( cluster_path / 'res_0-01_mask-MERIT-GLCLU2019_datasets-29_reruns-10_clusters-15.tif' )

In [6]:
# Define where the map needs to go
new_map = cluster_file.replace('.tif',f'_{tar_crs}.tif')

In [7]:
# Do the warping
options = gdal.WarpOptions(format='GTiff',
                           srcSRS='epsg:4326', dstSRS=tar_crs,
                           creationOptions=['COMPRESS=DEFLATE'])
gdal.Warp(new_map, cluster_file, options=options)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x10788c870> >

### Define the color scheme

Source: https://gis.stackexchange.com/a/325751

In [12]:
def set_geotiff_color_palette(file, map, band=1):
    
    # specify GeoTIFF file name, open it using GDAL and get the first band
    ds = gdal.Open(file, 1)
    band = ds.GetRasterBand(band)

    # create color table
    colors = gdal.ColorTable()

    # set color for each value
    if map == 1: #  res_0-01_datasets-29_clusters-15_reruns-10.tif
        colors.SetColorEntry( 0, (138, 140, 12))  # [1st map: 0] green/brown              
        colors.SetColorEntry( 1, (205, 241, 250)) # [1st map: 1] white-ish
        colors.SetColorEntry( 2, ( 28, 138, 227)) # [1st map: 2] medium blue
        colors.SetColorEntry( 3, (125, 250, 219)) # [1st map: 3] aquamarine
        colors.SetColorEntry( 4, (160, 241, 250)) # [1st map: 4] faded light blue
        colors.SetColorEntry( 5, (208,   3, 184)) # [1st map: 5] pink
        colors.SetColorEntry( 6, ( 11,  19, 238)) # [1st map: 6] dark blue, water
        colors.SetColorEntry( 7, ( 80, 140,   6)) # [1st map: 7] yellow/green
        colors.SetColorEntry( 8, (183, 203,  57)) # [1st map: 8] yellow
        colors.SetColorEntry( 9, ( 40, 213, 229)) # [1st map: 9] bright light blue
        colors.SetColorEntry(10, (208, 105,   9)) # [1st map: 10] orange
        colors.SetColorEntry(11, (212, 205, 183)) # [1st map: 11] pink
        colors.SetColorEntry(12, (230,  55,  11)) # [1st map: 12] red
        colors.SetColorEntry(13, (  5,  82,  13)) # [1st map: 13] dark green
        colors.SetColorEntry(14, (147, 208, 250)) # [1st map: 14] faded/bright light blue
    elif map == 2: #  res_0-01_mask-MERIT-GLCLU2019_datasets-29_reruns-10_clusters-15.ptif
        colors.SetColorEntry( 0, ( 91,  83,  41)) 
        colors.SetColorEntry( 1, ( 11,  19, 238)) 
        colors.SetColorEntry( 2, ( 76, 147, 213)) 
        colors.SetColorEntry( 3, (125, 250, 219)) 
        colors.SetColorEntry( 4, (  5,  82,  13)) 
        colors.SetColorEntry( 5, (212, 205, 183)) 
        colors.SetColorEntry( 6, (183, 203,  57)) 
        colors.SetColorEntry( 7, ( 72, 222, 115)) 
        colors.SetColorEntry( 8, (208, 105,   9)) 
        colors.SetColorEntry( 9, (147, 208, 250)) 
        colors.SetColorEntry(10, (138, 140,  12)) 
        colors.SetColorEntry(11, (176,  63, 202)) 
        colors.SetColorEntry(12, ( 80, 140,   6)) 
        colors.SetColorEntry(13, (205, 241, 250)) 
        colors.SetColorEntry(14, (160, 241, 250)) 
        

    # set color table and color interpretation
    band.SetRasterColorTable(colors)
    band.SetRasterColorInterpretation(gdal.GCI_PaletteIndex)

    # close and save file
    del band, ds

In [10]:
set_geotiff_color_palette(new_map, map=2)

In [11]:
set_geotiff_color_palette(cluster_file, map=2) # useful for vertical plots